In [1]:
spark.version

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7,application_1635758941805_0018,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res1: String = 3.0.1-amzn-0


In [2]:
val input_df = spark.read.parquet("s3://vasveena-test-demo/tmp/hudi-perf/input/")
input_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

input_df: org.apache.spark.sql.DataFrame = [id: bigint, month: bigint ... 5 more fields]
res2: Long = 2700000000


In [3]:
input_df.createOrReplaceTempView("inputdf")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
import org.apache.spark.sql.functions._

val input_df2=(input_df.withColumn("z", substring(md5(concat($"id")),1,1))
                       .withColumn("schema-v", lit("v1")).withColumn("data-v", lit("v2"))
                       .withColumn("trade_dt", substring($"modified_timestamp",1,10)))
input_df2.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.functions._
input_df2: org.apache.spark.sql.DataFrame = [id: bigint, month: bigint ... 9 more fields]
+-------+-----+-------+---+--------------------+----+-------------------+---+--------+------+----------+
|     id|month|     sk|txt|                uuid|year| modified_timestamp|  z|schema-v|data-v|  trade_dt|
+-------+-----+-------+---+--------------------+----+-------------------+---+--------+------+----------+
|4000000|    3|4000000|[E]|6e505939-f5fd-4ab...|2019|2021-04-02 00:05:02|  9|      v1|    v2|2021-04-02|
|4000001|    9|4000001|[F]|20486aca-2759-43f...|2019|2021-04-02 00:05:02|  d|      v1|    v2|2021-04-02|
|4000002|   11|4000002|[G]|42962a21-a2dc-40d...|2019|2021-04-02 00:05:02|  d|      v1|    v2|2021-04-02|
|4000003|    9|4000003|[H]|9841ad6d-1532-496...|2019|2021-04-02 00:05:02|  c|      v1|    v2|2021-04-02|
|4000004|    4|4000004|[I]|ff1a855a-cced-495...|2019|2021-04-02 00:05:02|  4|      v1|    v2|2021-04-02|
+-------+-----+-------+---+---

In [5]:
val input_df4 = input_df2.withColumnRenamed("schema-v", "schema_v").withColumnRenamed("data-v", "data_v")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

input_df4: org.apache.spark.sql.DataFrame = [id: bigint, month: bigint ... 9 more fields]


In [6]:
input_df4.printSchema

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: long (nullable = true)
 |-- month: long (nullable = true)
 |-- sk: long (nullable = true)
 |-- txt: struct (nullable = true)
 |    |-- key1: string (nullable = true)
 |-- uuid: string (nullable = true)
 |-- year: string (nullable = true)
 |-- modified_timestamp: timestamp (nullable = true)
 |-- z: string (nullable = true)
 |-- schema_v: string (nullable = false)
 |-- data_v: string (nullable = false)
 |-- trade_dt: string (nullable = true)



In [ ]:
spark.sql("drop table df_sparksql26")
spark.sql("drop table df_sparksql28")

In [11]:
spark.sql("""CREATE TABLE df_sparksql26 (id bigint,
                                       -- month bigint,
                                       sk bigint,
                                       txt struct<key1:string>,
                                       uuid string,
                                       year string,
                                       modified_timestamp timestamp)
PARTITIONED BY (z string,schema_v string,data_v string,trade_dt string)
STORED AS PARQUET
location  's3://vasveena-test-demo/iceberg/catalog/tables/db/df_sparksql26'""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res10: org.apache.spark.sql.DataFrame = []


In [12]:
spark.sql("""CREATE TABLE df_sparksql28 (id bigint,
                                       -- month bigint,
                                       sk bigint,
                                       txt struct<key1:string>,
                                       uuid string,
                                       year string,
                                       modified_timestamp timestamp)
PARTITIONED BY (z string,schema_v string,data_v string,trade_dt string)
STORED AS PARQUET
location  's3://vasveena-test-demo/iceberg/catalog/tables/db/df_sparksql28'""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res11: org.apache.spark.sql.DataFrame = []


In [9]:
spark.sql("set hive.exec.dynamic.partition.mode=nonstrict")
val t1 = System.nanoTime
input_df4.repartition(col("modified_timestamp"),col("month")).drop(col("month")).sortWithinPartitions("z","schema_v","data_v","trade_dt").write.mode("overwrite").insertInto("df_sparksql26")
val duration = (System.nanoTime - t1) / 1e9d + "seconds"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res8: org.apache.spark.sql.DataFrame = [key: string, value: string]
t1: Long = 214514296149693
duration: String = 727.337321312seconds


In [10]:
val t1 = System.nanoTime
input_df4.drop(col("month")).sortWithinPartitions("z","schema_v","data_v","trade_dt").write.mode("overwrite").insertInto("df_sparksql28")
val duration = (System.nanoTime - t1) / 1e9d + "seconds"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t1: Long = 215243749189459
duration: String = 440.433459461seconds


In [11]:
spark.sql("select count(*) from df_sparksql26").show()

VBox()

An error was encountered:
Invalid status code '404' from http://localhost:8998/sessions/3 with error payload: {"msg":"Session '3' not found."}


In [7]:
spark.sql("""CREATE TABLE df_sparksql22 (id bigint,
                                       -- month bigint,
                                       sk bigint,
                                       txt struct<key1:string>,
                                       uuid string,
                                       year string,
                                       modified_timestamp timestamp)
PARTITIONED BY (z string,schema_v string,data_v string,trade_dt string)
STORED AS PARQUET
location  's3://vasveena-test-demo/iceberg/catalog/tables/db/df_sparksql22'""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res7: org.apache.spark.sql.DataFrame = []


In [8]:
spark.sql("""CREATE TABLE df_sparksql24 (id bigint,
                                       -- month bigint,
                                       sk bigint,
                                       txt struct<key1:string>,
                                       uuid string,
                                       year string,
                                       modified_timestamp timestamp)
PARTITIONED BY (z string,schema_v string,data_v string,trade_dt string)
STORED AS PARQUET
location  's3://vasveena-test-demo/iceberg/catalog/tables/db/df_sparksql24'""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res8: org.apache.spark.sql.DataFrame = []


In [24]:
spark.sql("set spark.sql.parquet.binaryAsString=true")
//spark.sql("set spark.sql.hive.convertMetastoreParquet=false")
spark.sql("set hive.exec.dynamic.partition.mode=nonstrict")
val t1 = System.nanoTime
(input_df4.filter(input_df4("trade_dt") === "2021-04-02" 
                 && input_df4("data_v") === "v2" && input_df4("schema_v") === "v1" 
                 && input_df4("z") === "0")
         .repartition(col("modified_timestamp"),col("month")).drop(col("month"))
         .sortWithinPartitions("z","schema_v","data_v","trade_dt").write.mode("overwrite")
         .insertInto("df_sparksql22"))
val duration = (System.nanoTime - t1) / 1e9d + "seconds"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res41: org.apache.spark.sql.DataFrame = [key: string, value: string]
res43: org.apache.spark.sql.DataFrame = [key: string, value: string]
t1: Long = 290952087691310
duration: String = 232.564885904seconds


In [9]:
// Try repartition with partition field for better locality

//spark.sql("set spark.sql.parquet.binaryAsString=true")
//spark.sql("set spark.sql.hive.convertMetastoreParquet=false")
spark.sql("set hive.exec.dynamic.partition.mode=nonstrict")
val t1 = System.nanoTime
(input_df4.filter(input_df4("trade_dt") === "2021-04-02" 
                 && input_df4("data_v") === "v2" && input_df4("schema_v") === "v1" 
                 && input_df4("z") === "0")
         .repartition(400, col("z"),col("schema_v"),col("data_v"),col("trade_dt")).drop(col("month"))
         .sortWithinPartitions("z","schema_v","data_v","trade_dt").write.mode("overwrite")
         .insertInto("df_sparksql22"))
val duration = (System.nanoTime - t1) / 1e9d + "seconds"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res19: org.apache.spark.sql.DataFrame = [key: string, value: string]
res21: org.apache.spark.sql.DataFrame = [key: string, value: string]
t1: Long = 298681043645470
duration: String = 729.008731402seconds


In [8]:
spark.sql("set spark.sql.parquet.binaryAsString=true")
//spark.sql("set spark.sql.hive.convertMetastoreParquet=false")
spark.sql("set hive.exec.dynamic.partition.mode=nonstrict")
val t1 = System.nanoTime
(input_df4.filter(input_df4("trade_dt") === "2021-04-02" 
                 && input_df4("data_v") === "v2" && input_df4("schema_v") === "v1" 
                 && input_df4("z") === "0")
     .drop(col("month"))
     .sortWithinPartitions("z","schema_v","data_v","trade_dt")
     .write.mode("overwrite").insertInto("df_sparksql24"))
val duration = (System.nanoTime - t1) / 1e9d + "seconds"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res13: org.apache.spark.sql.DataFrame = [key: string, value: string]
res15: org.apache.spark.sql.DataFrame = [key: string, value: string]
t1: Long = 297723408524214
duration: String = 214.045636688seconds


In [ ]:
// Try repartition with partition field for better locality

spark.sql("set spark.sql.parquet.binaryAsString=true")
//spark.sql("set spark.sql.hive.convertMetastoreParquet=false")
spark.sql("set hive.exec.dynamic.partition.mode=nonstrict")
val t1 = System.nanoTime
(input_df4.filter(input_df4("trade_dt") === "2021-04-02" 
                 && input_df4("data_v") === "v2" && input_df4("schema_v") === "v1" 
                 && input_df4("z") === "0")
     .drop(col("month"))
     .sortWithinPartitions("z","schema_v","data_v","trade_dt")
     .write.mode("overwrite").insertInto("df_sparksql24"))
val duration = (System.nanoTime - t1) / 1e9d + "seconds"

In [9]:
spark.sql("""CREATE TABLE df_sparksql42 (id bigint,
                                       -- month bigint,
                                       sk bigint,
                                       txt struct<key1:string>,
                                       uuid string,
                                       year string,
                                       modified_timestamp timestamp)
PARTITIONED BY (z string,schema_v string,data_v string,trade_dt string)
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
location  's3://vasveena-test-demo/iceberg/catalog/tables/db/df_sparksql42'""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res10: org.apache.spark.sql.DataFrame = []


In [7]:
spark.sql("""CREATE TABLE df_sparksql44 (id bigint,
                                       -- month bigint,
                                       sk bigint,
                                       txt struct<key1:string>,
                                       uuid string,
                                       year string,
                                       modified_timestamp timestamp)
PARTITIONED BY (z string,schema_v string,data_v string,trade_dt string)
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
location  's3://vasveena-test-demo/iceberg/catalog/tables/db/df_sparksql44'""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res7: org.apache.spark.sql.DataFrame = []


In [10]:
spark.sql("set hive.exec.dynamic.partition.mode=nonstrict")
val t1 = System.nanoTime
input_df4.repartition(col("modified_timestamp"),col("month")).drop(col("month")).sortWithinPartitions("z","schema_v","data_v","trade_dt").write.mode("overwrite").insertInto("df_sparksql42")
val duration = (System.nanoTime - t1) / 1e9d + "seconds"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res11: org.apache.spark.sql.DataFrame = [key: string, value: string]
t1: Long = 283163222077723
duration: String = 1206.956524709seconds


In [13]:
val t1 = System.nanoTime
input_df4.drop(col("month")).sortWithinPartitions("z","schema_v","data_v","trade_dt").write.mode("overwrite").insertInto("df_sparksql44")
val duration = (System.nanoTime - t1) / 1e9d + "seconds"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t1: Long = 285813385453468
duration: String = 753.236217927seconds


In [16]:
spark.sql("set hive.exec.dynamic.partition.mode=nonstrict")
val t1 = System.nanoTime
input_df4.repartition(col("modified_timestamp"),col("month")).drop(col("month")).sortWithinPartitions("z","schema_v","data_v","trade_dt").write.mode("overwrite").parquet("s3://vasveena-test-demo/iceberg/catalog/tables/db/df_sparksql46")
val duration = (System.nanoTime - t1) / 1e9d + "seconds"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res20: org.apache.spark.sql.DataFrame = [key: string, value: string]
t1: Long = 287758440216721
duration: String = 670.769490376seconds


In [17]:
spark.sql("set hive.exec.dynamic.partition.mode=nonstrict")
val t1 = System.nanoTime
input_df4.drop(col("month")).sortWithinPartitions("z","schema_v","data_v","trade_dt").write.mode("overwrite").parquet("s3://vasveena-test-demo/iceberg/catalog/tables/db/df_sparksql48")
val duration = (System.nanoTime - t1) / 1e9d + "seconds"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res22: org.apache.spark.sql.DataFrame = [key: string, value: string]
t1: Long = 288429606857936
duration: String = 346.258263351seconds
